# Relax Inc. Take Home Challenge
## *The goal is to identify which factors predict future user adoption*

In [1]:
# imports
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

/home/aregel/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# load data into dataframes
user_engagement = pd.read_csv('takehome_user_engagement.csv')
users = pd.read_csv('takehome_users.csv', encoding = 'iso-8859-1')

In [3]:
# change dates to datetime objects
users['creation_time'] = pd.to_datetime(users.creation_time)
users['last_session_creation_time'] = pd.to_datetime(users.last_session_creation_time)
user_engagement['time_stamp'] = pd.to_datetime(user_engagement.time_stamp)

### Identify  users who have adopted the service

In [4]:
# create dictionary where user_id is key, and list of timestamps are value
usage = defaultdict(list)
for ID, time in zip(user_engagement.user_id, user_engagement.time_stamp):
    usage[ID].append(time)

In [5]:
def within_7days(lst):
    '''takes a list of logins and determines if 2 or more occur in 7 days  '''
    delta_days = [] # list of 2 logins that happend withing 7 days
    for num in range(0, len(lst)-1):
        A = lst[num]
        B = lst[num + 1]
        delta_days.append(np.abs(A - B))
        
    final_lst = [day for day in delta_days if day < timedelta(days = 7)]
    if len(final_lst) > 0:
        return 1
    else:
        return 0
    

In [6]:
user_adoption = defaultdict(int)
for ID, logs in usage.items():
    if len(logs) < 2:
        user_adoption[ID] = 0
    else:
        user_adoption[ID] = within_7days(logs)
        
    

In [7]:
users['adoption'] = [user_adoption[ID] for ID in users.object_id]

### Use logistic regression to model the data

In [8]:
# Identify features and target
X = users[['creation_source', 'opted_in_to_mailing_list', 'enabled_for_marketing_drip', 'org_id']]
y = users.adoption

In [9]:
# one-hot encode categorical variables
X = pd.get_dummies(X, columns = ['creation_source', 'org_id'], drop_first=True)

In [10]:
# split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [11]:
# create and fit logistic regression model
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [12]:
# validate with test data
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_pred, y_test)
print("The logisic regression model has an accuracy of {}".format(round(accuracy,3)))


The logisic regression model has an accuracy of 0.822


In [13]:
# determine which features are the most important
i = 0
features = []
for feature in X.columns.values:
    features.append((clf.coef_[0][i], feature))
    i += 1
#print the top 10
sorted(features, reverse=True)[:10]

[(1.3156117049462588, 'org_id_387'),
 (1.0934500813735186, 'org_id_352'),
 (0.9967893890191881, 'org_id_58'),
 (0.9715008235616759, 'org_id_231'),
 (0.9708260076832462, 'org_id_270'),
 (0.9548530405734894, 'org_id_156'),
 (0.947318097925644, 'org_id_273'),
 (0.9335049630301615, 'org_id_381'),
 (0.9018968250377211, 'org_id_161'),
 (0.8874086423107751, 'org_id_295')]

## It is clear to see from the coefficent values of the logistic regression, that the  orgainization the group of users belongs to is the most important factor in user adoption.